# Health Data Outcomes using Linear Regression Project IV

**Grupo** <br />
Rafael Guimarães <br />
Bruno Monteiro   <br />
Harrison Santos  <br />


**Disciplina:** Métodos Numéricos <br />
**Professor:** Paulo Ribeiro

In [70]:
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math, scipy, numpy as np
from scipy import linalg
import warnings
warnings.filterwarnings('ignore')

O conjuto de dados escolhido para a replicação do metodo de regressão linear [[1]](https://nbviewer.jupyter.org/github/fastai/numerical-linear-algebra/blob/master/nbs/5.%20Health%20Outcomes%20with%20Linear%20Regression.ipynb#5.-Health-Outcomes-with-Linear-Regression) está disponível na documentação do sklearn [[2]](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html) referente ao conjunto de dados do cancêr de mama, em Wisconsin. USA. 

In [71]:
data = datasets.load_breast_cancer()

In [72]:
feature_names = ['mean radius', 'mean texture', 'mean perimeter', 'mean area',
        'mean smoothness', 'mean compactness', 'mean concavity',
        'mean concave points', 'mean symmetry', 'mean fractal dimension',
        'radius error', 'texture error', 'perimeter error', 'area error',
        'smoothness error', 'compactness error', 'concavity error',
        'concave points error', 'symmetry error',
        'fractal dimension error', 'worst radius', 'worst texture',
        'worst perimeter', 'worst area', 'worst smoothness',
        'worst compactness', 'worst concavity', 'worst concave points',
        'worst symmetry', 'worst fractal dimension']

In [73]:
trn,test,y_trn,y_test = train_test_split(data.data, data.target, test_size=0.2)

In [74]:
trn.shape, test.shape

((455, 30), (114, 30))

# Regressão linear no Scikit Learn

Considerando um sistema $$X\beta = y$$, em que $X$ possui mais linhas que colunas. Isso ocorre quando você tem mais amostras de dados do que variáveis. Queremos encontrar β que minimize: 


$$∣∣X\beta − y∣∣_{2}$$ 

Vamos executar a implementação do sklearn:

In [75]:
regr = linear_model.LinearRegression()
%timeit regr.fit(trn, y_trn)

960 µs ± 7.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [76]:
pred = regr.predict(test)

A seguir veremos algumas métricas sobre o quão boa é nossa previsão. Veremos a norma quadrática média (L2) e o erro absoluto médio (L1).

In [77]:
def regr_metrics(act, pred):
    return (math.sqrt(metrics.mean_squared_error(act, pred)), 
     metrics.mean_absolute_error(act, pred))

In [78]:
regr_metrics(y_test, regr.predict(test))

(0.2745650835026591, 0.2146580502783168)

In [79]:
trn.shape

(455, 30)

In [80]:
poly = PolynomialFeatures(include_bias=False)

In [81]:
trn_feat = poly.fit_transform(trn)

In [82]:
', '.join(poly.get_feature_names(feature_names))

'mean radius, mean texture, mean perimeter, mean area, mean smoothness, mean compactness, mean concavity, mean concave points, mean symmetry, mean fractal dimension, radius error, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst perimeter, worst area, worst smoothness, worst compactness, worst concavity, worst concave points, worst symmetry, worst fractal dimension, mean radius^2, mean radius mean texture, mean radius mean perimeter, mean radius mean area, mean radius mean smoothness, mean radius mean compactness, mean radius mean concavity, mean radius mean concave points, mean radius mean symmetry, mean radius mean fractal dimension, mean radius radius error, mean radius texture error, mean radius perimeter error, mean radius area error, mean radius smoothness error, mean radius compactness error, mean radius concavity error, mean radius c

In [83]:
trn_feat.shape

(455, 495)

In [84]:
regr.fit(trn_feat, y_trn)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [85]:
regr_metrics(y_test, regr.predict(poly.fit_transform(test)))

(11.37553249805324, 2.5185843777359804)

In [86]:
%timeit poly.fit_transform(trn)

11.7 ms ± 61.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Experimentos com vetorização e códigos nativos

In [87]:
%matplotlib inline

In [88]:
import math, numpy as np, matplotlib.pyplot as plt
from pandas_summary import DataFrameSummary
from scipy import ndimage

In [89]:
from numba import jit, vectorize, guvectorize, cuda, float32, void, float64

De acordo com [1] todas a utilização dessas bibliotecas vão acelerar o processo de vetorização do método de regressão

In [90]:
def proc_python(xx,yy):
    zz = np.zeros(nobs, dtype='float32')
    for j in range(nobs):   
        x, y = xx[j], yy[j] 
        x = x*2 - ( y * 55 )
        y = x + y*2         
        z = x + y + 99      
        z = z * ( z - .88 ) 
        zz[j] = z           
    return zz

In [91]:
nobs = 10000
x = np.random.randn(nobs).astype('float32')
y = np.random.randn(nobs).astype('float32')

In [92]:
%timeit proc_python(x,y)   # Untyped and unvectorized

174 ms ± 6.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [93]:
# Typed and Vectorized
def proc_numpy(x,y):
    z = np.zeros(nobs, dtype='float32')
    x = x*2 - ( y * 55 )
    y = x + y*2         
    z = x + y + 99      
    z = z * ( z - .88 ) 
    return z

In [94]:
np.allclose( proc_numpy(x,y), proc_python(x,y), atol=1e-4 )

True

In [95]:
%timeit proc_numpy(x,y)    # Typed and vectorized

67.8 µs ± 8.95 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Conforme foi demonstrado em [1] podemos comprovar aqui uma diferença considerável em termos de desempenho quando o processo de procura em dados **vetorizados** versus **não vetorizados** 

Enquanto no primeiro caso tivemos 

166 ms ± 1.87 por loop

No segundo usando 

~~~python
def proc_numpy(x,y): # vectorized
~~~

Tivemos 

60.5 µs ± 744 ns por loop

Agora vamos testar usando **Numba**

In [96]:
@jit()
def proc_numba(xx,yy,zz):
    for j in range(nobs):   
        x, y = xx[j], yy[j] 
        x = x*2 - ( y * 55 )
        y = x + y*2         
        z = x + y + 99      
        z = z * ( z - .88 ) 
        zz[j] = z           
    return zz

In [97]:
z = np.zeros(nobs).astype('float32')
np.allclose( proc_numpy(x,y), proc_numba(x,y,z), atol=1e-4 )

True

In [98]:
%timeit proc_numba(x,y,z)

8.38 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Como podemos vizualizar o compilador do Numba otimiza o código de uma maneira mais inteligente do que é possível com Python e Numpy.

In [99]:
@vectorize
def vec_numba(x,y):
    x = x*2 - ( y * 55 )
    y = x + y*2         
    z = x + y + 99      
    return z * ( z - .88 ) 

In [100]:
np.allclose(vec_numba(x,y), proc_numba(x,y,z), atol=1e-4 )

True

In [101]:
%timeit vec_numba(x,y)

8.36 µs ± 20.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [102]:
# Funções polinomiais com Numba

@jit(nopython=True)
def vec_poly(x, res):
    m,n=x.shape
    feat_idx=0
    for i in range(n):
        v1=x[:,i]
        for k in range(m): res[k,feat_idx] = v1[k]
        feat_idx+=1
        for j in range(i,n):
            for k in range(m): res[k,feat_idx] = v1[k]*x[k,j]
            feat_idx+=1

In [103]:
trn = np.asfortranarray(trn)
test = np.asfortranarray(test)

In [104]:
m,n=trn.shape
n_feat = n*(n+1)//2 + n
trn_feat = np.zeros((m,n_feat), order='F')
test_feat = np.zeros((len(y_test), n_feat), order='F')

In [105]:
vec_poly(trn, trn_feat)
vec_poly(test, test_feat)

In [106]:
regr.fit(trn_feat, y_trn)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [107]:
regr_metrics(y_test, regr.predict(test_feat))

(11.375541605536771, 2.5185851433818294)

In [108]:
%timeit vec_poly(trn, trn_feat)

206 µs ± 3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [109]:
%timeit poly.fit_transform(trn)

11.9 ms ± 331 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Regularização e ruído

A regularização é uma maneira de reduzir o excesso de ajuste e criar modelos que generalizem melhor para novos dados.

In [110]:
reg_regr = linear_model.LassoCV(n_alphas=10)

In [111]:
reg_regr.fit(trn_feat, y_trn)

LassoCV(alphas=None, copy_X=True, cv='warn', eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=10, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [112]:
reg_regr.alpha_

484.93662302137415

In [113]:
regr_metrics(y_test, reg_regr.predict(test_feat))

(0.2793076860180617, 0.22432416842595687)

# Ruído

Agora vamos adicionar algum ruído aos dados

In [114]:
idxs = np.random.randint(0, len(trn), 10)

In [115]:
y_trn2 = np.copy(y_trn)
y_trn2[idxs] *= 10 # label noise

In [116]:
regr = linear_model.LinearRegression()
regr.fit(trn, y_trn)
regr_metrics(y_test, regr.predict(test))

(0.2745650835026575, 0.21465805027831458)

In [117]:
regr.fit(trn, y_trn2)
regr_metrics(y_test, regr.predict(test))

(0.4050552616858666, 0.3045026155157403)

In [119]:
hregr = linear_model.HuberRegressor()
hregr.fit(trn, y_trn2)
regr_metrics(y_test, hregr.predict(test))

(0.3452927761520907, 0.25015459075174973)

De acordo com [1] a perda de huber é uma função de perda menos sensível a valores discrepantes do que a perda de erro ao quadrado. É quadrático para valores de erro pequenos e linear para valores grandes.